# INFO 3350/6350

## Lecture 02(b): Pandas

Pandas is the general-purpose data library for Python. Its most important data type is the `DataFrame`. DataFrames are two-dimensional arrays of arbitrary types that have a bunch of convenient indexing, slicing, and mathematical manipulation functions. 

There's a sense in which pandas is a combination of python dictionaries (fast, convenient indexing and lookups) with numpy arrays (fast math). But don't overthink it. We're pandas users, not pandas developers.

A little more simply, a typical DataFrame looks like this:

In [ ]:
import pandas as pd

book_dict =  {
    'author': ['Shakespeare', 'Morrison', 'Bolaño'],
    'title': ['King Lear', 'Beloved', '2666'],
    'year': [1606, 1987, 2004],
    'words': [10000, 100000, 300000]
}

book_df = pd.DataFrame(book_dict) # <- note that we're using the dict defined above
book_df

Notice that this DataFrame consists of **rows** and **columns**. It might remind you of a spreadsheet or a database table.

You can think of a **row** as a **record** or an **observation**. It contains all the information about an *individual object*.

A **column** contains the data about a **single feature** for **all objects** in the DataFrame.

Our DataFrame has three rows (books) and four columns (features of those books).

Note the special pseudo-column at the left, the **index**.

In [ ]:
# columns have types
book_df.dtypes

In [ ]:
# more info about our dataframe
book_df.info()

In [ ]:
# select a column, return a series
book_df['author']

Think of this like a Python dictionary, where columns are keyed by name.

A Series is similar to a DataFrame, but has strictly one dimension. This is sometimes convenient, but is also sometimes the source of subtle bugs. When in doubt, select Pandas columns using **two** sets of square brackets, which returns a DataFrame, even if there's only one column in it.

Let's **do** something with a column:

In [ ]:
# column-wise (vectorized) operations
book_df[['year']]+100

We **didn't** need to iterate over the elements in the column. The mathematical operation was applied automatically to very element. These kinds of vectorized operations are often much faster than loops. We'll try to use them whenever we can.

The above code returned a DataFrame. If we had left one of the sets of square brackets out, we would have returned a Pandas Series:

In [ ]:
# return a series, not a frame
book_df['year']+100

In [ ]:
# did we modify the underlying dataframe?
book_df

In [ ]:
# indexing
new_df = book_df.loc[1:2, ['year', 'title']]
new_df

The general form here is `.loc[row_indexer, column_indexer]`. We'll have more to say about this in a minute. Here, notice that the row indexer is operating on labels, **not index positions**, and that it is inclusive. To index by positions, use `.iloc` (which behaves like regular python indexing, hence does not include the terminal value).

We can also manipulate the index, which isn't a column like the others.

In [ ]:
# set an existing column to be the index
book_df.set_index(['author'], inplace=True)
book_df

In [ ]:
# select using the new index
book_df.loc['Morrison', 'title']

In [ ]:
book_df['title']

In [ ]:
# pandas tries to infer what you meant, but can fail
# when in doubt, be explicit
# here, get all indices (rows) and just the 'title' column
book_df.loc[:,['title']]

In [ ]:
book_df

In [ ]:
# modify the underlying dataframe
book_df['year'] = book_df['year']+100
book_df

In [ ]:
# create a new column based on an existing one
book_df['old_year'] = book_df['year']-100
book_df

In [ ]:
# you can call the index itself
book_df.index

In [ ]:
# but you can't select an index by name
book_df[['author']]

In [ ]:
# for reference, can get values of a named index
book_df.index.get_level_values('author')

## Indexing, slicing, and setting values

Dataframes are valuable mostly because they make it easy to select and group data. Let's take a look at how this works.

In [ ]:
d = {
    'a':[1,2,3],
    'b':[4,5,6]
}
df = pd.DataFrame(d)
df

In [ ]:
# Add a column
df['c'] = [True, True, False] # a boolean column

In [ ]:
df

In [ ]:
# get the value at row 0, column b
df.loc[0,'b'] # format is row label, column label

In [ ]:
# alternatively, when we want a SINGLE value only
df.at[0, 'b']

In [ ]:
# Make the index confusing
df.index = [5,18,3]
df

In [ ]:
# Indices are labels, NOT positions!
df.loc[18:3]

In [ ]:
# Select by index position
df.iloc[[1]]

In [ ]:
# Select using boolean array
df.loc[[True, True, False]]

In [ ]:
# did not modify df
df

Selections are (usually) **views** into the existing dataframe. They do not (usually) create new dataframes. This is good and efficient, but can sometimes be tricky ...

### Set value on copy of slice warning

As we just saw, you can select rows using `.loc`. It's tempting to then select columns from the resulting subset of rows. This is called chained indexing. There's nothing inherently wrong with chaining, but it can be the source of downstream problems. To wit:

#### The wrong way!

In [ ]:
# Select rows ...
subset = df[ df['b']<=5 ]
subset

In [ ]:
# How does this work?
df['b'] <= 5

So, we pass a boolean array in as a row selector, then ...

In [ ]:
# Then select columns from the subset and try to modify their values
subset['c'] = False

Why does this happen? Why throw a warning here?

In [ ]:
# What was the output?
display(subset) # FYI, note use of display() IPython function

In [ ]:
# Compare the original dataframe
df

Eep! This **did not** modify the original dataframe. Did we want it to? Pandas doesn't know, and the outcome is unpredictable (it depends on certain low-level memory layout features that Pandas doesn't control). Hence, the warning.

#### The correct way

Be explicit using `loc` and do it in a single operation!

In [ ]:
# From the warning: "Try using .loc[row_indexer,col_indexer] = value instead"
df.loc[ df['b']<=5, 'c' ] = False
df

In [ ]:
# or we can use explicit .copy()
subset = subset.copy() # <- copy the data, so it's disassociated from the source df
display(subset)
subset['c'] = True
display(subset)

`.copy()`ing can be slow when you have a lot of data. Use with care.

## Working with real data

An example using COVID data from Tompkins County.

Note that **you** can use Sheets to share data for your group. Pull directly to Pandas as CSV over the web by inserting `/export?format=csv` between the document identifier and any `&gid=` sheet identifier in the URL. Unrestricted link sharing must be turned on.

For example:

```
https://docs.google.com/spreadsheets/d/1_EYNs90DIWYg9huCiRlw4H2FqofIhDutburi9RrUMYY/edit?usp=sharing
                                                                                    ^^^^^^^^^^^^^^^^
```

... becomes ...

```
https://docs.google.com/spreadsheets/d/1_EYNs90DIWYg9huCiRlw4H2FqofIhDutburi9RrUMYY/export?format=csv
                                                                                    ^^^^^^^^^^^^^^^^^
```

Or, you can use Google's [Python API library](https://developers.google.com/sheets/api/quickstart/python) for authenticated access (but it's probably overkill).

### Data prep

Read the [awkwardly formatted data table](https://docs.google.com/spreadsheets/d/e/2PACX-1vQvvugFsb4GePXQnmEZbgrtqmJRiaA7tO1UGSBwvBdhbJEmf2ntzE0am-x-Lo6mLPj9ASLpAg6UZsCF/pubhtml?gid=1214476126&single=true) over the web. Take a look at this and discuss what you expect on import (problems, data types, dataframe length and shape, etc.).

In [ ]:
# Get data from TCHD's Google sheet
sheet = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQvvugFsb4GePXQnmEZbgrtqmJRiaA7tO1UGSBwvBdhbJEmf2ntzE0am-x-Lo6mLPj9ASLpAg6UZsCF/pubhtml?gid=1214476126&single=true'
fetched_data = pd.read_html(sheet, skiprows=1)
covid = fetched_data[0]
covid.head()

In [ ]:
# working with .columns
print(covid.columns)

In [ ]:
# get rid of undesired data
columns_to_retain = list(covid.columns)[1:] # discard first column
#columns_to_retain.pop()                     # discard last column
covid = covid.loc[1:, columns_to_retain]   # discard empty first row
covid.head()

In [ ]:
# Write out the good data to disk for later reuse
covid.to_csv('lec-02-tompkins-covid-data.csv', index=False)

### Read in cleaned data

First, examine the pure CSV using a text editor, then come back here.

In [ ]:
covid = pd.read_csv('lec-02-tompkins-covid-data.csv')
covid.head()

What is `NaN`?

### Change a column name

In [ ]:
# option 1
col_names = list(covid.columns)
col_names[6] = 'active_cases'
covid.columns = col_names
covid.head()

In [ ]:
# option 2
covid.rename(columns={'Active COVID-19 Hospitalizations (CHS)':'active_hospitalizations'}, inplace=True)
covid.head()

### Explore the data

In [ ]:
# types
covid.dtypes

Why these data types? (Compare input data from sheet ...)

In [ ]:
# table size
covid.shape

In [ ]:
# just table length
len(covid)

In [ ]:
# get descriptive stats
covid.describe()

### Change column types

In [ ]:
# 'Date' column has type 'object' (often used for strings)
covid['Date'].dtype

In [ ]:
# take one example date entry - it's a string
type(covid.at[0, 'Date'])

In [ ]:
# convert Date column to datetime type
covid['Date'] = pd.to_datetime(covid['Date'])
covid[['Date']].head()

In [ ]:
covid['Date'].dtype

In [ ]:
# take one example date entry - it's now a Timestamp
type(covid.at[0, 'Date'])

Datetime objects are useful because we can do convenient selection, sampling, and slicing with them. More about this next time ...

In [ ]:
# plot directly from Pandas
covid[['Date', 'active_cases']].set_index(['Date']).plot();